<a href="https://colab.research.google.com/github/RusAl84/text_mining/blob/master/%D0%9C%D0%B0%D1%81%D1%82%D0%B5%D1%80_%D0%BA%D0%BB%D0%B0%D1%81%D1%81_%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%B8%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82%D1%83%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TF-IDF

У подхода bag-of-words есть существенный недостаток. Если слово встречается 5 раз в конкретном документе, но и в других документах тоже встречается часто, то его наличие в документе не особо-то о чём-то говорит. Если же слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения bag-of-words различий не будет: в обеих ячейках будет просто число 5.

Отчасти это решается исключением стоп-слов (и слишком часто встречающихся слов), но лишь отчасти. Другой идеей является отмасштабировать получившуюся таблицу с учётом "редкости" слова в наборе документов (т.е. с учётом информативности слова).

tfidf=tf∗idf

idf=log((N+1)/(Nw+1))+1

Здесь tf это частота слова в тексте (то же самое, что в bag of words), N - общее число документов, Nw - число документов, содержащих данное слово.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

В sklearn есть класс для поддержки TF-IDF: TfidfVectorizer, рассмотрим его.

In [ ]:
texts = "\u0427\u0442\u043E \u0442\u0430\u043A\u043E\u0435 \u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439? What is IoT and IoT Security.jpg \u041C\u0438\u0440 \u0441\u0442\u0430\u043B \u0446\u0438\u0444\u0440\u043E\u0432\u044B\u043C. \u0421\u043E\u0442\u043E\u0432\u044B\u0435 \u0442\u0435\u043B\u0435\u0444\u043E\u043D\u044B \u0442\u0435\u043F\u0435\u0440\u044C \u043E\u0431\u044B\u0447\u043D\u043E\u0435 \u0434\u0435\u043B\u043E, \u0432 \u0448\u043A\u043E\u043B\u0430\u0445 \u043F\u043B\u0430\u043D\u0448\u0435\u0442\u044B \u0437\u0430\u043C\u0435\u043D\u0438\u043B\u0438 \u0442\u0435\u0442\u0440\u0430\u0434\u0438, \u0430 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438 \u0440\u0430\u0437\u0440\u0430\u0431\u0430\u0442\u044B\u0432\u0430\u044E\u0442 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u043D\u043E\u0432\u043E\u0433\u043E \u043F\u043E\u043A\u043E\u043B\u0435\u043D\u0438\u044F, \u043D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u0430\u0432\u0442\u043E\u043C\u043E\u0431\u0438\u043B\u0438 \u0431\u0435\u0437 \u0432\u043E\u0434\u0438\u0442\u0435\u043B\u044F.  \u041A\u0430\u0436\u0435\u0442\u0441\u044F, \u0432\u0441\u0435 \u0441\u0432\u044F\u0437\u0430\u043D\u043E \u043C\u0435\u0436\u0434\u0443 \u0441\u043E\u0431\u043E\u0439, \u043E\u0441\u043E\u0431\u0435\u043D\u043D\u043E \u0432 \u0431\u0438\u0437\u043D\u0435\u0441\u0435. \u0411\u0443\u0434\u044C \u0442\u043E \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0441\u0438\u0441\u0442\u0435\u043C\u044B \u0431\u0435\u0437\u043E\u043F\u0430\u0441\u043D\u043E\u0441\u0442\u0438 \u0438\u043B\u0438 \u043D\u043E\u0443\u0442\u0431\u0443\u043A\u0438 - \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432, \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043D\u044B\u0445 \u043A \u0441\u0435\u0442\u0438 \u0438 \u0440\u0430\u0431\u043E\u0442\u0430\u044E\u0449\u0438\u0445 \u0432\u043C\u0435\u0441\u0442\u0435, \u0442\u043E\u043B\u044C\u043A\u043E \u0440\u0430\u0441\u0442\u0435\u0442. \u041F\u043E \u043F\u043E\u0441\u043B\u0435\u0434\u043D\u0438\u043C \u0434\u0430\u043D\u043D\u044B\u043C, \u043A 2020 \u0433\u043E\u0434\u0443 \u043A \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0443 \u0431\u0443\u0434\u0435\u0442 \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043E \u043E\u043A\u043E\u043B\u043E 20,4 \u043C\u0438\u043B\u043B\u0438\u0430\u0440\u0434\u0430 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432. \u0414\u043B\u044F \u0442\u0430\u043A\u043E\u0433\u043E \u0440\u043E\u0434\u0430 \xAB\u0432\u0437\u0430\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u044F\xBB \u0435\u0441\u0442\u044C \u0441\u043F\u0435\u0446\u0438\u0430\u043B\u044C\u043D\u044B\u0439 \u0442\u0435\u0440\u043C\u0438\u043D - \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439 (Internet of Things - IoT).  \u0425\u043E\u0442\u044F \u0442\u0435\u0440\u043C\u0438\u043D \u0443\u0436\u0435 \u0432 \u043E\u0431\u0438\u0445\u043E\u0434\u0435 \u0431\u043E\u043B\u044C\u0448\u0435 10 \u043B\u0435\u0442, \u043D\u0435 \u0432\u0441\u0435 \u0434\u043E \u0441\u0438\u0445 \u043F\u043E\u0440 \u043F\u043E\u043B\u043D\u043E\u0441\u0442\u044C\u044E \u043F\u043E\u043D\u0438\u043C\u0430\u044E\u0442, \u0447\u0442\u043E \u043D\u0430 \u0441\u0430\u043C\u043E\u043C \u0434\u0435\u043B\u0435 \u043E\u043D \u043E\u0437\u043D\u0430\u0447\u0430\u0435\u0442 \u0438 \u043F\u043E\u0447\u0435\u043C\u0443 \u043E\u043D \u0442\u0430\u043A \u0432\u0430\u0436\u0435\u043D \u0434\u043B\u044F \u0431\u0438\u0437\u043D\u0435\u0441\u0430 \u0438 \u043F\u043E\u0442\u0440\u0435\u0431\u0438\u0442\u0435\u043B\u0435\u0439.   \u0427\u0442\u043E \u0442\u0430\u043A\u043E\u0435 \"\u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\"? \u0422\u0435\u0440\u043C\u0438\u043D \"\u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\" (IoT) - \u043E\u0437\u043D\u0430\u0447\u0430\u0435\u0442 \u0432\u0437\u0430\u0438\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432, \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043D\u044B\u0445 \u043A \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0443 \u0438 \"\u043E\u0431\u0449\u0430\u044E\u0449\u0438\u0445\u0441\u044F\" \u043C\u0435\u0436\u0434\u0443 \u0441\u043E\u0431\u043E\u0439. \u0412\u044B, \u0432\u0435\u0440\u043E\u044F\u0442\u043D\u043E, \u0441\u0435\u0439\u0447\u0430\u0441 \u043F\u043E\u0434\u0443\u043C\u0430\u043B\u0438 \u043E \u0442\u0430\u043A\u0438\u0445 \u0432\u0435\u0449\u0430\u0445, \u043A\u0430\u043A \u043D\u043E\u0443\u0442\u0431\u0443\u043A \u0438\u043B\u0438 \u0443\u043C\u043D\u044B\u0439 \u0442\u0435\u043B\u0435\u0432\u0438\u0437\u043E\u0440, \u043D\u043E IoT \u043E\u0445\u0432\u0430\u0442\u044B\u0432\u0430\u0435\u0442 \u0437\u043D\u0430\u0447\u0438\u0442\u0435\u043B\u044C\u043D\u043E \u0431\u043E\u043B\u0435\u0435 \u0440\u0430\u0437\u043D\u043E\u043E\u0431\u0440\u0430\u0437\u043D\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430, \u043D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u044D\u043B\u0435\u043A\u0442\u0440\u043E\u043D\u0438\u043A\u0443, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u043D\u0438\u043A\u043E\u0433\u0434\u0430 \u0440\u0430\u043D\u0435\u0435 \u043D\u0435 \u0431\u044B\u043B\u0430 \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u0430 \u043A \u0441\u0435\u0442\u0438 - \u0434\u043E\u043C\u0430\u0448\u043D\u0438\u0435 \u0445\u043E\u043B\u043E\u0434\u0438\u043B\u044C\u043D\u0438\u043A\u0438, \u043A\u043E\u0444\u0435\u043C\u0430\u0448\u0438\u043D\u044B, \u0441\u0442\u0438\u0440\u0430\u043B\u044C\u043D\u044B\u0435 \u043C\u0430\u0448\u0438\u043D\u044B \u0438 \u0442\u0430\u043A \u0434\u0430\u043B\u0435\u0435. \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439 - \u044D\u0442\u043E \u0432\u0441\u0435 (\u0434\u0430\u0436\u0435 \u043D\u0435\u043E\u0431\u044B\u0447\u043D\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430), \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043C\u043E\u0433\u0443\u0442 \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0430\u0442\u044C\u0441\u044F \u043A \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0443. \u0412 \u043D\u0430\u0448\u0438 \u0434\u043D\u0438 \u043F\u043E\u0447\u0442\u0438 \u043B\u044E\u0431\u043E\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u043E \u0441 \u0432\u044B\u043A\u043B\u044E\u0447\u0430\u0442\u0435\u043B\u0435\u043C \u043F\u043E\u0442\u0435\u043D\u0446\u0438\u0430\u043B\u044C\u043D\u043E \u043C\u043E\u0436\u0435\u0442 \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0430\u0442\u044C\u0441\u044F \u043A \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0443, \u0447\u0442\u043E \u0434\u0435\u043B\u0430\u0435\u0442 \u0435\u0433\u043E \u0447\u0430\u0441\u0442\u044C\u044E IoT.  \u041F\u043E\u0447\u0435\u043C\u0443 \u0432\u0441\u0435 \u0441\u0435\u0433\u043E\u0434\u043D\u044F \u0433\u043E\u0432\u043E\u0440\u044F\u0442 \u043E\u0431 \"\u0438\u043D\u0435\u0440\u043D\u0435\u0442\u0435 \u0432\u0435\u0449\u0435\u0439? \"\u0418\u043D\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\" - \u0430\u043A\u0442\u0443\u0430\u043B\u044C\u043D\u0430\u044F \u0442\u0435\u043C\u0430 \u043D\u0430 \u0441\u0435\u0433\u043E\u0434\u043D\u044F\u0448\u043D\u0438\u0439 \u0434\u0435\u043D\u044C, \u043F\u043E\u0442\u043E\u043C\u0443 \u0447\u0442\u043E \u043C\u044B \u043E \u043F\u043E\u043D\u0438\u043C\u0430\u0435\u043C, \u0441\u043A\u043E\u043B\u044C\u043A\u043E \u0432\u0435\u0449\u0435\u0439 \u043C\u043E\u0436\u0435\u0442 \u0431\u044B\u0442\u044C \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043E \u043A \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0443 \u0438 \u043A\u0430\u043A \u044D\u0442\u043E \u043C\u043E\u0436\u0435\u0442 \u043F\u043E\u0432\u043B\u0438\u044F\u0442\u044C \u043D\u0430 \u043F\u043E\u0432\u0441\u0435\u0434\u043D\u0435\u0432\u043D\u0443\u044E \u0436\u0438\u0437\u043D\u044C. \u0420\u0430\u0437\u0432\u0438\u0442\u0438\u0435 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0439 \u0437\u0430 \u043F\u043E\u0441\u043B\u0435\u0434\u043D\u0435\u0435 \u0434\u0435\u0441\u044F\u0442\u0438\u043B\u0435\u0442\u0438\u0435 \u0441\u043F\u043E\u0441\u043E\u0431\u0441\u0442\u0432\u043E\u0432\u0430\u043B\u043E \u0442\u0430\u043A\u0438\u043C \u0444\u0430\u043A\u0442\u043E\u0440\u0430\u043C:  \u0421\u043F\u043E\u0441\u043E\u0431\u044B \u0441\u043E\u0437\u0434\u0430\u043D\u0438\u044F \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438 \xAB\u0443\u043C\u043D\u044B\u0445\xBB \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432 \u0441\u0442\u0430\u043B\u0438 \u0431\u043E\u043B\u0435\u0435 \u0434\u043E\u0441\u0442\u0443\u043F\u043D\u044B\u043C\u0438  \u0412\u043E\u0437\u0440\u043E\u0441\u043B\u043E \u0447\u0438\u0441\u043B\u043E Wi-Fi - \u0441\u043E\u0432\u043C\u0435\u0441\u0442\u0438\u043C\u044B\u0445 \u043F\u0440\u043E\u0434\u0443\u043A\u0442\u043E\u0432 \u041A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0435\u0439 \u0441\u043C\u0430\u0440\u0442\u0444\u043E\u043D\u0430\u043C\u0438 \u0432\u043E \u0432\u0441\u0435\u043C \u043C\u0438\u0440\u0443 \u0440\u0435\u0437\u043A\u043E \u0443\u0432\u0435\u043B\u0438\u0447\u0438\u043B\u043E\u0441\u044C \u041F\u043E\u044F\u0432\u0438\u043B\u0438\u0441\u044C \u0432\u043E\u0437\u043C\u043E\u0436\u043D\u043E\u0441\u0442\u0438 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u044C \u0441\u043C\u0430\u0440\u0442\u0444\u043E\u043D \u043A\u0430\u043A \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u043E, \u0443\u043F\u0440\u0430\u0432\u043B\u044F\u044E\u0449\u0435\u0435 \u0434\u0440\u0443\u0433\u0438\u043C\u0438 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430\u043C\u0438 \u0418\u043C\u0435\u043D\u043D\u043E \u043F\u043E\u044D\u0442\u043E\u043C\u0443 IoT - \u0431\u043E\u043B\u044C\u0448\u0435 \u043D\u0435 \u043F\u0440\u043E\u0444\u0435\u0441\u0441\u0438\u043E\u043D\u0430\u043B\u044C\u043D\u044B\u0439 IT-\u0441\u043B\u0435\u043D\u0433. \u042D\u0442\u043E \u0442\u0435\u0440\u043C\u0438\u043D, \u043A\u043E\u0442\u043E\u0440\u044B\u0439 \u0434\u043E\u043B\u0436\u0435\u043D \u0437\u043D\u0430\u0442\u044C \u0432 \u043F\u0435\u0440\u0432\u0443\u044E \u043E\u0447\u0435\u0440\u0435\u0434\u044C \u043A\u0430\u0436\u0434\u044B\u0439 \u0432\u043B\u0430\u0434\u0435\u043B\u0435\u0446 \u0431\u0438\u0437\u043D\u0435\u0441\u0430.  \u041F\u0440\u0438\u043A\u043B\u0430\u0434\u043D\u043E\u0435 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043D\u0438\u0435 \"\u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0430 \u0432\u0435\u0449\u0435\u0439\". \u041A\u0430\u043A\u0438\u0435 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F \u043D\u0430\u0438\u0431\u043E\u043B\u0435\u0435 \u0447\u0430\u0441\u0442\u043E \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u044E\u0442\u0441\u044F \u043D\u0430 \u0440\u0430\u0431\u043E\u0447\u0435\u043C \u043C\u0435\u0441\u0442\u0435? \u0418\u0441\u0441\u043B\u0435\u0434\u043E\u0432\u0430\u043D\u0438\u044F \u043F\u043E\u043A\u0430\u0437\u044B\u0432\u0430\u044E\u0442, \u0447\u0442\u043E IoT- \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u0441\u043F\u043E\u0441\u043E\u0431\u0441\u0442\u0432\u0443\u044E\u0442 \u043F\u043E\u0432\u044B\u0448\u0435\u043D\u0438\u044E \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u0438 \u0440\u0430\u0431\u043E\u0442\u044B \u043F\u0440\u0435\u0434\u043F\u0440\u0438\u044F\u0442\u0438\u044F. \u041F\u0440\u043E\u0438\u0437\u0432\u043E\u0434\u0438\u0442\u0435\u043B\u044C\u043D\u043E\u0441\u0442\u044C \u0442\u0440\u0443\u0434\u0430 \u0441\u043E\u0442\u0440\u0443\u0434\u043D\u0438\u043A\u043E\u0432, \u0434\u0438\u0441\u0442\u0430\u043D\u0446\u0438\u043E\u043D\u043D\u044B\u0439 \u043C\u043E\u043D\u0438\u0442\u043E\u0440\u0438\u043D\u0433 \u0438 \u043E\u0442\u043B\u0430\u0436\u0435\u043D\u043D\u044B\u0435 \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u044B \u044F\u0432\u043B\u044F\u044E\u0442\u0441\u044F \u043E\u0434\u043D\u0438\u043C\u0438 \u0438\u0437 \u0433\u043B\u0430\u0432\u043D\u044B\u0445 \u043F\u0440\u0435\u0438\u043C\u0443\u0449\u0435\u0441\u0442\u0432 IoT \u0434\u043B\u044F \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0439.  \u041D\u043E \u043A\u0430\u043A \u0436\u0435 \u0432\u044B\u0433\u043B\u044F\u0434\u0438\u0442 \u0438\u043D\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439 \u0432\u043D\u0443\u0442\u0440\u0438 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438? \u041A\u0430\u0436\u0434\u0430\u044F \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u044F \u0438\u043C\u0435\u0435\u0442 \u0441\u0432\u043E\u0438 \u043E\u0441\u043E\u0431\u0435\u043D\u043D\u043E\u0441\u0442\u0438, \u043D\u043E \u0432\u043E\u0442 \u043D\u0435\u0441\u043A\u043E\u043B\u044C\u043A\u043E \u043F\u0440\u0438\u043C\u0435\u0440\u043E\u0432 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043D\u0438\u044F IoT \u043D\u0430 \u0440\u0430\u0431\u043E\u0442\u0435:  \u0423\u043C\u043D\u044B\u0435 \u0437\u0430\u043C\u043A\u0438 \u043F\u043E\u0437\u0432\u043E\u043B\u044F\u044E\u0442 \u0440\u0443\u043A\u043E\u0432\u043E\u0434\u0438\u0442\u0435\u043B\u044E \u043F\u0440\u0435\u0434\u043F\u0440\u0438\u044F\u0442\u0438\u044F \u043E\u0442\u043A\u0440\u044B\u0432\u0430\u0442\u044C \u0434\u0432\u0435\u0440\u044C \u0441\u043E \u0441\u0432\u043E\u0435\u0433\u043E \u0441\u043C\u0430\u0440\u0442\u0444\u043E\u043D\u0430, \u043F\u0440\u043E\u043F\u0443\u0441\u043A\u0430\u044F \u0441\u043E\u0442\u0440\u0443\u0434\u043D\u0438\u043A\u0430 \u043D\u0430 \u0440\u0430\u0431\u043E\u0447\u0435\u0435 \u043C\u0435\u0441\u0442\u043E \u0432 \u0441\u0443\u0431\u0431\u043E\u0442\u0443. \u0418\u043D\u0442\u0435\u043B\u043B\u0435\u043A\u0442\u0443\u0430\u043B\u044C\u043D\u044B\u0435 \u0442\u0435\u0440\u043C\u043E\u0441\u0442\u0430\u0442\u044B \u0438 \u0444\u043E\u043D\u0430\u0440\u0438 \u0432\u043A\u043B\u044E\u0447\u0430\u044E\u0442\u0441\u044F \u0438 \u0432\u044B\u043A\u043B\u044E\u0447\u0430\u044E\u0442\u0441\u044F \u0432 \u043D\u0443\u0436\u043D\u043E\u0435 \u0432\u0440\u0435\u043C\u044F, \u0447\u0442\u043E\u0431\u044B \u0441\u044D\u043A\u043E\u043D\u043E\u043C\u0438\u0442\u044C \u043D\u0430 \u0440\u0430\u0441\u0445\u043E\u0434\u0430\u0445 \u043D\u0430 \u044D\u043B\u0435\u043A\u0442\u0440\u043E\u044D\u043D\u0435\u0440\u0433\u0438\u044E. \u0413\u043E\u043B\u043E\u0441\u043E\u0432\u044B\u0435 \u043F\u043E\u043C\u043E\u0449\u043D\u0438\u043A\u0438, \u0442\u0430\u043A\u0438\u0435 \u043A\u0430\u043A Siri \u0438\u043B\u0438 Alexa, \u043E\u0442\u043A\u0440\u044B\u0432\u0430\u044E\u0442 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043F\u043E\u0437\u0432\u043E\u043B\u044F\u044E\u0442 \u0432\u0430\u043C, \u043D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u0434\u0435\u043B\u0430\u0442\u044C \u0437\u0430\u043C\u0435\u0442\u043A\u0438, \u0443\u0441\u0442\u0430\u043D\u0430\u0432\u043B\u0438\u0432\u0430\u0442\u044C \u043D\u0430\u043F\u043E\u043C\u0438\u043D\u0430\u043D\u0438\u044F, \u043F\u043E\u043B\u0443\u0447\u0430\u0442\u044C \u0434\u043E\u0441\u0442\u0443\u043F \u043A \u0441\u0432\u043E\u0435\u043C\u0443 \u043A\u0430\u043B\u0435\u043D\u0434\u0430\u0440\u044E \u0438\u043B\u0438 \u043E\u0442\u043F\u0440\u0430\u0432\u043B\u044F\u0442\u044C \u044D\u043B\u0435\u043A\u0442\u0440\u043E\u043D\u043D\u044B\u0435 \u043F\u0438\u0441\u044C\u043C\u0430. \u041F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043D\u044B\u0435 \u0434\u0430\u0442\u0447\u0438\u043A\u0438 \u0432\u043D\u0443\u0442\u0440\u0438 \u043F\u0440\u0438\u043D\u0442\u0435\u0440\u043E\u0432 \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u044F\u044E\u0442 \u043D\u0438\u0437\u043A\u0438\u0439 \u0443\u0440\u043E\u0432\u0435\u043D\u044C \u0442\u043E\u043D\u0435\u0440\u0430 \u0438 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u0438 \u0440\u0430\u0437\u043C\u0435\u0449\u0430\u044E\u0442 \u0437\u0430\u043A\u0430\u0437 \u043D\u0430 \u0435\u0433\u043E \u043F\u043E\u043F\u043E\u043B\u043D\u0435\u043D\u0438\u0435. \u041A\u0430\u043C\u0435\u0440\u044B \u0432\u0438\u0434\u0435\u043E\u043D\u0430\u0431\u043B\u044E\u0434\u0435\u043D\u0438\u044F, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043F\u043E\u0437\u0432\u043E\u043B\u044F\u044E\u0442 \u0442\u0440\u0430\u043D\u0441\u043B\u0438\u0440\u043E\u0432\u0430\u0442\u044C \u043A\u043E\u043D\u0442\u0435\u043D\u0442 \u0447\u0435\u0440\u0435\u0437 \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442. \u0417\u0430\u0447\u0435\u043C \u043D\u0443\u0436\u043D\u043E \u0437\u043D\u0430\u0442\u044C \u043E \u0431\u0435\u0437\u043E\u043F\u0430\u0441\u043D\u043E\u0441\u0442\u0438 \u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0430 \u0432\u0435\u0449\u0435\u0439? \u041F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043D\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u043C\u043E\u0433\u0443\u0442 \u0437\u043D\u0430\u0447\u0438\u0442\u0435\u043B\u044C\u043D\u043E \u043F\u043E\u0432\u044B\u0441\u0438\u0442\u044C \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u044C \u0432\u0430\u0448\u0435\u0433\u043E \u0431\u0438\u0437\u043D\u0435\u0441\u0430, \u043D\u043E \u0432\u0441\u0435, \u0447\u0442\u043E \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043E \u043A \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0443, \u043C\u043E\u0436\u0435\u0442 \u0431\u044B\u0442\u044C \u0443\u044F\u0437\u0432\u0438\u043C\u043E \u0434\u043B\u044F \u043A\u0438\u0431\u0435\u0440\u0430\u0442\u0430\u043A. \u041A\u0438\u0431\u0435\u0440\u043F\u0440\u0435\u0441\u0442\u0443\u043F\u043D\u0438\u043A\u0438 \u043C\u043E\u0433\u0443\u0442 \u043D\u0430\u0439\u0442\u0438 \u0441\u043F\u043E\u0441\u043E\u0431 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u044C \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044E, \u0441\u043E\u0431\u0440\u0430\u043D\u043D\u0443\u044E \u043D\u0430 \u0440\u0430\u0437\u043D\u044B\u0445 \u0442\u043E\u0447\u043A\u0430\u0445 \u0432\u043D\u0443\u0442\u0440\u0438 IoT-\u044D\u043A\u043E\u0441\u0438\u0441\u0442\u0435\u043C\u044B - \u043E\u0442 \u043A\u043E\u0440\u043F\u043E\u0440\u0430\u0442\u0438\u0432\u043D\u044B\u0445 \u0441\u0435\u0440\u0432\u0435\u0440\u043E\u0432 \u0434\u043E \u043E\u0431\u043B\u0430\u0447\u043D\u043E\u0433\u043E \u0445\u0440\u0430\u043D\u0438\u043B\u0438\u0449\u0430. \u042D\u0442\u043E \u043D\u0435 \u0437\u043D\u0430\u0447\u0438\u0442, \u0447\u0442\u043E \u0432\u044B \u0434\u043E\u043B\u0436\u043D\u044B \u0441\u0440\u043E\u0447\u043D\u043E \u0437\u0430\u043C\u0435\u043D\u0438\u0442\u044C \u0441\u0432\u043E\u0439 \u0440\u0430\u0431\u043E\u0447\u0438\u0439 \u043F\u043B\u0430\u043D\u0448\u0435\u0442 \u043D\u0430 \u0440\u0443\u0447\u043A\u0443 \u0438 \u0431\u0443\u043C\u0430\u0433\u0443. \u042D\u0442\u043E \u0437\u043D\u0430\u0447\u0438\u0442, \u0447\u0442\u043E \u0432\u044B \u0434\u043E\u043B\u0436\u043D\u044B \u0441\u0435\u0440\u044C\u0435\u0437\u043D\u043E \u043E\u0442\u043D\u043E\u0441\u0438\u0442\u044C\u0441\u044F \u043A \u0431\u0435\u0437\u043E\u043F\u0430\u0441\u043D\u043E\u0441\u0442\u0438 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0430 \u0432\u0435\u0449\u0435\u0439. \u0412\u043E\u0442 \u043D\u0435\u0441\u043A\u043E\u043B\u044C\u043A\u043E \u0441\u043E\u0432\u0435\u0442\u043E\u0432:  \u0414\u0435\u043B\u0430\u0439\u0442\u0435 \u0432\u043A\u043B\u0430\u0434\u043A\u0438 \u043D\u0430 \u043C\u043E\u0431\u0438\u043B\u044C\u043D\u044B\u0445 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430\u0445  \u0423\u0431\u0435\u0434\u0438\u0442\u0435\u0441\u044C, \u0447\u0442\u043E \u043C\u043E\u0431\u0438\u043B\u044C\u043D\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430, \u043D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u043F\u043B\u0430\u043D\u0448\u0435\u0442\u044B \u0440\u0435\u0433\u0438\u0441\u0442\u0440\u0438\u0440\u0443\u044E\u0442\u0441\u044F \u0438 \u0431\u043B\u043E\u043A\u0438\u0440\u0443\u044E\u0442\u0441\u044F \u0432 \u043A\u043E\u043D\u0446\u0435 \u043A\u0430\u0436\u0434\u043E\u0433\u043E \u0440\u0430\u0431\u043E\u0447\u0435\u0433\u043E \u0434\u043D\u044F. \u0415\u0441\u043B\u0438 \u043F\u043B\u0430\u043D\u0448\u0435\u0442 \u0443\u0442\u0435\u0440\u044F\u043D, \u0434\u0430\u043D\u043D\u044B\u0435 \u0438 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044F \u043C\u043E\u0433\u0443\u0442 \u0441\u0442\u0430\u0442\u044C \u0434\u043E\u0441\u0442\u0443\u043F\u043D\u044B\u043C\u0438 \u0437\u043B\u043E\u0443\u043C\u044B\u0448\u043B\u0435\u043D\u043D\u0438\u043A\u0430\u043C \u0438 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043D\u044B \u0438\u043C\u0438 \u0432 \u043D\u0435\u043D\u0430\u0434\u043B\u0435\u0436\u0430\u0449\u0438\u0445 \u0446\u0435\u043B\u044F\u0445. \u041E\u0431\u044F\u0437\u0430\u0442\u0435\u043B\u044C\u043D\u043E \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u0439\u0442\u0435 \u043D\u0430\u0434\u0435\u0436\u043D\u044B\u0439 \u043F\u0430\u0440\u043E\u043B\u044C \u0438\u043B\u0438 \u0431\u0438\u043E\u043C\u0435\u0442\u0440\u0438\u0447\u0435\u0441\u043A\u0438\u0439 \u043F\u0430\u0440\u043E\u043B\u044C, \u0447\u0442\u043E\u0431\u044B \u043D\u0438\u043A\u0442\u043E \u043D\u0435 \u043C\u043E\u0433 \u0432\u043E\u0439\u0442\u0438 \u0432 \u0432\u0430\u0448\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u043E, \u0435\u0441\u043B\u0438 \u043E\u043D\u043E \u0443\u0442\u0435\u0440\u044F\u043D\u043E \u0438\u043B\u0438 \u0443\u043A\u0440\u0430\u0434\u0435\u043D\u043E. \u0418\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u0439\u0442\u0435 \u0437\u0430\u0449\u0438\u0442\u043D\u044B\u0439 \u043F\u0440\u043E\u0434\u0443\u043A\u0442, \u043F\u043E\u0437\u0432\u043E\u043B\u044F\u044E\u0449\u0438\u0439 \u043E\u0433\u0440\u0430\u043D\u0438\u0447\u0438\u0432\u0430\u0442\u044C \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0431\u0443\u0434\u0443\u0442 \u0440\u0430\u0431\u043E\u0442\u0430\u0442\u044C \u043D\u0430 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0435, \u0440\u0430\u0437\u0434\u0435\u043B\u044F\u0442\u044C \u0434\u0435\u043B\u043E\u0432\u044B\u0435 \u0438 \u043B\u0438\u0447\u043D\u044B\u0435 \u0434\u0430\u043D\u043D\u044B\u0435 \u0438 \u0441\u0442\u0438\u0440\u0430\u0442\u044C \u043A\u043E\u0440\u043F\u043E\u0440\u0430\u0442\u0438\u0432\u043D\u0443\u044E \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044E \u0432 \u0441\u043B\u0443\u0447\u0430\u0435 \u043A\u0440\u0430\u0436\u0438 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430. \u0423\u0441\u0442\u0430\u043D\u043E\u0432\u0438\u0442\u0435 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u043E\u0431\u043D\u043E\u0432\u043B\u0435\u043D\u0438\u044F \u0430\u043D\u0442\u0438\u0432\u0438\u0440\u0443\u0441\u0430  \u0412\u0430\u043C \u043D\u0443\u0436\u043D\u043E \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u043D\u043E\u0435 \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u0435 \u0434\u043B\u044F \u0432\u0441\u0435\u0445 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432 \u0434\u043B\u044F \u0437\u0430\u0449\u0438\u0442\u044B \u043E\u0442 \u0432\u0438\u0440\u0443\u0441\u043E\u0432, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043F\u0440\u0435\u0434\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u044E\u0442 \u0445\u0430\u043A\u0435\u0440\u0430\u043C \u0434\u043E\u0441\u0442\u0443\u043F \u043A \u0432\u0430\u0448\u0435\u0439 \u0441\u0438\u0441\u0442\u0435\u043C\u0435 \u0438 \u0434\u0430\u043D\u043D\u044B\u043C. \u041D\u0430\u0441\u0442\u0440\u043E\u0439\u0442\u0435 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u043E\u0431\u043D\u043E\u0432\u043B\u0435\u043D\u0438\u044F \u0430\u043D\u0442\u0438\u0432\u0438\u0440\u0443\u0441\u0430 \u0434\u043B\u044F \u0437\u0430\u0449\u0438\u0442\u044B \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432 \u043E\u0442 \u043A\u0438\u0431\u0435\u0440\u0430\u0442\u0430\u043A\u0438. \u0422\u0440\u0435\u0431\u0443\u0439\u0442\u0435 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043D\u0438\u044F \u043D\u0430\u0434\u0435\u0436\u043D\u044B\u0445 \u043B\u043E\u0433\u0438\u043D\u043E\u0432 \u0434\u043B\u044F \u0432\u0445\u043E\u0434\u0430  \u041C\u043D\u043E\u0433\u0438\u0435 \u043B\u044E\u0434\u0438 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u044E\u0442 \u043E\u0434\u0438\u043D\u0430\u043A\u043E\u0432\u044B\u0435 \u043B\u043E\u0433\u0438\u043D \u0438 \u043F\u0430\u0440\u043E\u043B\u044C \u043D\u0430 \u043A\u0430\u0436\u0434\u043E\u043C \u0438\u0437 \u0441\u0432\u043E\u0438\u0445 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432. \u0422\u0430\u043A \u0438\u0445 \u043B\u0435\u0433\u0447\u0435 \u0437\u0430\u043F\u043E\u043C\u043D\u0438\u0442\u044C. \u0410 \u043A\u0438\u0431\u0435\u0440\u043F\u0440\u0435\u0441\u0442\u0443\u043F\u043D\u0438\u043A\u0430\u043C \u043B\u0435\u0433\u0447\u0435 \u0432\u0437\u043B\u043E\u043C\u0430\u0442\u044C. \u0423\u0431\u0435\u0434\u0438\u0442\u0435\u0441\u044C, \u0447\u0442\u043E \u043A\u0430\u0436\u0434\u044B\u0439 \u043B\u043E\u0433\u0438\u043D \u0443\u043D\u0438\u043A\u0430\u043B\u0435\u043D \u0434\u043B\u044F \u043A\u0430\u0436\u0434\u043E\u0433\u043E \u0441\u043E\u0442\u0440\u0443\u0434\u043D\u0438\u043A\u0430 \u0438 \u0442\u0440\u0435\u0431\u0443\u0439\u0442\u0435 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043D\u0438\u044F \u043D\u0430\u0434\u0435\u0436\u043D\u044B\u0445 \u043F\u0430\u0440\u043E\u043B\u0435\u0439. \u0412\u0441\u0435\u0433\u0434\u0430 \u043C\u0435\u043D\u044F\u0439\u0442\u0435 \u043F\u0430\u0440\u043E\u043B\u044C \u043F\u043E \u0443\u043C\u043E\u043B\u0447\u0430\u043D\u0438\u044E \u043D\u0430 \u043D\u043E\u0432\u044B\u0445 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430\u0445. \u041D\u0438\u043A\u043E\u0433\u0434\u0430 \u043D\u0435 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u0439\u0442\u0435 \u043E\u0434\u0438\u043D \u0438 \u0442\u043E\u0442 \u0436\u0435 \u043F\u0430\u0440\u043E\u043B\u044C \u043D\u0430 \u0440\u0430\u0437\u043D\u044B\u0445 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430\u0445. \u0420\u0430\u0437\u0432\u0435\u0440\u043D\u0438\u0442\u0435 \u0441\u043A\u0432\u043E\u0437\u043D\u043E\u0435 \u0448\u0438\u0444\u0440\u043E\u0432\u0430\u043D\u0438\u0435  \u041F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043D\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u043E\u0431\u0449\u0430\u044E\u0442\u0441\u044F \u0434\u0440\u0443\u0433 \u0441 \u0434\u0440\u0443\u0433\u043E\u043C, \u0438 \u043A\u043E\u0433\u0434\u0430 \u043E\u043D\u0438 \u044D\u0442\u043E \u0434\u0435\u043B\u0430\u044E\u0442, \u0434\u0430\u043D\u043D\u044B\u0435 \u043F\u0435\u0440\u0435\u0434\u0430\u044E\u0442\u0441\u044F \u043E\u0442 \u043E\u0434\u043D\u043E\u0433\u043E \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u043A \u0434\u0440\u0443\u0433\u043E\u043C\u0443. \u0412\u0430\u043C \u043D\u0443\u0436\u043D\u043E \u0437\u0430\u0448\u0438\u0444\u0440\u043E\u0432\u044B\u0432\u0430\u0442\u044C \u0434\u0430\u043D\u043D\u044B\u0435 \u0432 \u043A\u0430\u0436\u0434\u043E\u0439 \u0442\u043E\u0447\u043A\u0435 \u043F\u0435\u0440\u0435\u0441\u0435\u0447\u0435\u043D\u0438\u044F. \u0414\u0440\u0443\u0433\u0438\u043C\u0438 \u0441\u043B\u043E\u0432\u0430\u043C\u0438, \u0432\u0430\u043C \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u043E \u0441\u043A\u0432\u043E\u0437\u043D\u043E\u0435 \u0448\u0438\u0444\u0440\u043E\u0432\u0430\u043D\u0438\u0435 \u0434\u043B\u044F \u0437\u0430\u0449\u0438\u0442\u044B \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u0438 \u0432\u043E \u0432\u0440\u0435\u043C\u044F \u0435\u0435 \u043F\u0435\u0440\u0435\u043C\u0435\u0449\u0435\u043D\u0438\u044F \u043E\u0442 \u0442\u043E\u0447\u043A\u0438 \u043A \u0442\u043E\u0447\u043A\u0435. \u0423\u0431\u0435\u0434\u0438\u0442\u0435\u0441\u044C, \u0447\u0442\u043E \u043E\u0431\u043D\u043E\u0432\u043B\u0435\u043D\u0438\u044F \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u0438 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u043D\u043E\u0433\u043E \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F \u0434\u043E\u0441\u0442\u0443\u043F\u043D\u044B, \u0438 \u0443\u0441\u0442\u0430\u043D\u0430\u0432\u043B\u0438\u0432\u0430\u0439\u0442\u0435 \u0438\u0445 \u0432\u043E\u0432\u0440\u0435\u043C\u044F  \u041F\u0440\u0438 \u043F\u043E\u043A\u0443\u043F\u043A\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u043E \u0443\u0431\u0435\u0434\u0438\u0442\u044C\u0441\u044F, \u0447\u0442\u043E \u043F\u0440\u043E\u0438\u0437\u0432\u043E\u0434\u0438\u0442\u0435\u043B\u044C \u043F\u0440\u0435\u0434\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442 \u043E\u0431\u043D\u043E\u0432\u043B\u0435\u043D\u0438\u044F. \u0423\u0441\u0442\u0430\u043D\u0430\u0432\u043B\u0438\u0432\u0430\u0439\u0442\u0435 \u043E\u0431\u043D\u043E\u0432\u043B\u0435\u043D\u0438\u044F, \u043A\u0430\u043A \u0442\u043E\u043B\u044C\u043A\u043E \u043E\u043D\u0438 \u0441\u0442\u0430\u043D\u043E\u0432\u044F\u0442\u0441\u044F \u0434\u043E\u0441\u0442\u0443\u043F\u043D\u044B. \u041F\u043E \u0432\u043E\u0437\u043C\u043E\u0436\u043D\u043E\u0441\u0442\u0438 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u0439\u0442\u0435 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u043E\u0431\u043D\u043E\u0432\u043B\u0435\u043D\u0438\u044F, \u043A\u0430\u043A \u0443\u043A\u0430\u0437\u0430\u043D\u043E \u0432\u044B\u0448\u0435. \u041E\u0442\u0441\u043B\u0435\u0436\u0438\u0432\u0430\u0439\u0442\u0435 \u0434\u043E\u0441\u0442\u0443\u043F\u043D\u044B\u0435 \u0444\u0443\u043D\u043A\u0446\u0438\u0438 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u0438 \u043E\u0442\u043A\u043B\u044E\u0447\u0430\u0439\u0442\u0435 \u0444\u0443\u043D\u043A\u0446\u0438\u0438, \u043A\u043E\u0442\u043E\u0440\u044B\u043C\u0438 \u043D\u0435 \u043F\u043E\u043B\u044C\u0437\u0443\u0435\u0442\u0435\u0441\u044C  \u041F\u0440\u043E\u0432\u0435\u0440\u044C\u0442\u0435 \u0434\u043E\u0441\u0442\u0443\u043F\u043D\u044B\u0435 \u0444\u0443\u043D\u043A\u0446\u0438\u0438 \u043D\u0430 \u0441\u0432\u043E\u0438\u0445 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430\u0445 \u0438 \u043E\u0442\u043A\u043B\u044E\u0447\u0438\u0442\u0435 \u0442\u0435, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0432\u044B \u043D\u0435 \u0441\u043E\u0431\u0438\u0440\u0430\u0435\u0442\u0435\u0441\u044C \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u044C, \u0447\u0442\u043E\u0431\u044B \u0443\u043C\u0435\u043D\u044C\u0448\u0438\u0442\u044C \u043F\u043E\u0442\u0435\u043D\u0446\u0438\u0430\u043B\u044C\u043D\u044B\u0435 \u0432\u043E\u0437\u043C\u043E\u0436\u043D\u043E\u0441\u0442\u0438 \u0430\u0442\u0430\u043A\u0438. \u0412\u044B\u0431\u0435\u0440\u0438\u0442\u0435 \u043E\u043F\u044B\u0442\u043D\u043E\u0433\u043E \u043F\u043E\u0441\u0442\u0430\u0432\u0449\u0438\u043A\u0430 \u0440\u0435\u0448\u0435\u043D\u0438\u0439 \u043A\u0438\u0431\u0435\u0440\u0431\u0435\u0437\u043E\u043F\u0430\u0441\u043D\u043E\u0441\u0442\u0438  \u0412\u044B \u0445\u043E\u0442\u0438\u0442\u0435, \u0447\u0442\u043E\u0431\u044B \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0430 \u0432\u0435\u0449\u0435\u0439 \u0441\u043F\u043E\u0441\u043E\u0431\u0441\u0442\u0432\u043E\u0432\u0430\u043B \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u044E \u0432\u0430\u0448\u0435\u0433\u043E \u0431\u0438\u0437\u043D\u0435\u0441\u0430, \u0430 \u043D\u0435 \u0432\u0440\u0435\u0434\u0438\u043B\u0430 \u0435\u043C\u0443. \u041E\u0431\u0440\u0430\u0442\u0438\u0442\u0435\u0441\u044C \u0437\u0430 \u043F\u043E\u043C\u043E\u0449\u044C\u044E \u043A \u0430\u0432\u0442\u043E\u0440\u0438\u0442\u0435\u0442\u043D\u043E\u043C\u0443 \u043F\u043E\u0441\u0442\u0430\u0432\u0449\u0438\u043A\u0443 \u0440\u0435\u0448\u0435\u043D\u0438\u0439 \u043A\u0438\u0431\u0435\u0440\u0431\u0435\u0437\u043E\u043F\u0430\u0441\u043D\u043E\u0441\u0442\u0438 \u0438 \u0430\u043D\u0442\u0438\u0432\u0438\u0440\u0443\u0441\u043D\u043E\u0439 \u0437\u0430\u0449\u0438\u0442\u044B, \u043A\u043E\u0442\u043E\u0440\u044B\u0439 \u0438\u043C\u0435\u0435\u0442 \u043E\u043F\u044B\u0442 \u0431\u043E\u0440\u044C\u0431\u044B \u0441 \u0443\u044F\u0437\u0432\u0438\u043C\u043E\u0441\u0442\u044F\u043C\u0438 \u0438 \u043F\u0440\u0435\u0434\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442 \u0443\u043D\u0438\u043A\u0430\u043B\u044C\u043D\u044B\u0435 \u0440\u0435\u0448\u0435\u043D\u0438\u044F \u0434\u043B\u044F \u043F\u0440\u0435\u0434\u043E\u0442\u0432\u0440\u0430\u0449\u0435\u043D\u0438\u044F \u043A\u0438\u0431\u0435\u0440\u0430\u0442\u0430\u043A. \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439 \u0431\u043E\u043B\u044C\u0448\u0435 \u043D\u0435 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0447\u0435\u0441\u043A\u043E\u0435 \u0443\u0432\u043B\u0435\u0447\u0435\u043D\u0438\u0435. \u0412\u0441\u0435 \u0431\u043E\u043B\u044C\u0448\u0435 \u043E\u0440\u0433\u0430\u043D\u0438\u0437\u0430\u0446\u0438\u0439 \u0440\u0435\u0430\u043B\u0438\u0437\u043E\u0432\u044B\u0432\u0430\u044E\u0442 \u0441\u0432\u043E\u0439 \u043F\u043E\u0442\u0435\u043D\u0446\u0438\u0430\u043B, \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u044F \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043D\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430. \u041D\u043E \u043E \u0431\u0435\u0437\u043E\u043F\u0430\u0441\u043D\u043E\u0441\u0442\u0438 \u0437\u0430\u0431\u044B\u0432\u0430\u0442\u044C \u043D\u0435\u043B\u044C\u0437\u044F. \u041A\u043E\u0433\u0434\u0430 \u0432\u044B \u0441\u043E\u0437\u0434\u0430\u0435\u0442\u0435 \u0441\u0432\u043E\u044E IoT-\u044D\u043A\u043E\u0441\u0438\u0441\u0442\u0435\u043C\u0443, \u0443\u0431\u0435\u0434\u0438\u0442\u0435\u0441\u044C, \u0447\u0442\u043E \u0432\u0430\u0448\u0430 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u044F, \u0434\u0430\u043D\u043D\u044B\u0435 \u0438 \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u044B \u0437\u0430\u0449\u0438\u0449\u0435\u043D\u044B.   \u0421\u0441\u044B\u043B\u043A\u0438 \u043F\u043E \u0442\u0435\u043C\u0435: \u0417\u0430\u0449\u0438\u0442\u0430 \u0432\u0441\u0442\u0440\u0430\u0438\u0432\u0430\u0435\u043C\u044B\u0445 \u0441\u0438\u0441\u0442\u0435\u043C \u0438 \u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0430 \u0432\u0435\u0449\u0435\u0439  \u0420\u0435\u0448\u0435\u043D\u0438\u044F \u0434\u043B\u044F \u0437\u0430\u0449\u0438\u0442\u044B \u043A\u0440\u0443\u043F\u043D\u044B\u0445 \u043F\u0440\u0435\u0434\u043F\u0440\u0438\u044F\u0442\u0438\u0439" #@param {type:"string"}
stexts = texts
print(texts)

Что такое интернет вещей? What is IoT and IoT Security.jpg Мир стал цифровым. Сотовые телефоны теперь обычное дело, в школах планшеты заменили тетради, а компании разрабатывают технологии нового поколения, например, автомобили без водителя.  Кажется, все связано между собой, особенно в бизнесе. Будь то автоматические системы безопасности или ноутбуки - количество устройств, подключенных к сети и работающих вместе, только растет. По последним данным, к 2020 году к Интернету будет подключено около 20,4 миллиарда устройств. Для такого рода «взамодействия» есть специальный термин - Интернет вещей (Internet of Things - IoT).  Хотя термин уже в обиходе больше 10 лет, не все до сих пор полностью понимают, что на самом деле он означает и почему он так важен для бизнеса и потребителей.   Что такое "интернет вещей"? Термин "интернет вещей" (IoT) - означает взаимодействие устройств, подключенных к Интернету и "общающихся" между собой. Вы, вероятно, сейчас подумали о таких вещах, как ноутбук или у

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [ ]:
def printTF_IDF(texts):
  records_count = 30
  tfIdfTransformer = TfidfVectorizer(ngram_range=(1, 4), use_idf=True, max_features=records_count)
  countVectorizer = CountVectorizer(ngram_range=(1, 4), max_features=records_count)
  wordCount = countVectorizer.fit_transform([texts])
  TfIdf = tfIdfTransformer.fit_transform([texts])
  names = countVectorizer.get_feature_names()
  df=[]
  df = pd.DataFrame(list(names), columns=['names'])
  df = df.assign(Word_Count=wordCount.T.todense())
  df = df.assign(TF_IDF=TfIdf.T.todense())
  df = df.sort_values('TF_IDF', ascending=False)
  print(df)

In [ ]:
printTF_IDF(texts)

              names  Word_Count    TF_IDF
17               на          17  0.386763
1             вещей          15  0.341262
28              что          14  0.318511
0               iot          12  0.273009
26       устройства          12  0.273009
5               для          12  0.273009
18               не          10  0.227508
25        устройств           7  0.159256
9          интернет           7  0.159256
14              как           7  0.159256
13        интернету           6  0.136505
20       обновления           6  0.136505
15          которые           6  0.136505
29              это           6  0.136505
10   интернет вещей           6  0.136505
2               все           6  0.136505
8               или           6  0.136505
3                вы           6  0.136505
11        интернета           5  0.113754
4            данные           5  0.113754
22           пароль           5  0.113754
12  интернета вещей           5  0.113754
19               но           5  0

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


удалим знаки переноса строк и сделаем текст в нижнем регистре

In [ ]:
  str2 = ''
  for item in texts.split():
    str2 = str2 + ' ' + item
  texts = str(texts)
  texts = texts.lower()
  # print(texts)
  texts = texts.replace('\n', ' ')

удалим цифры

In [ ]:
  str2 = ''
  for c in texts:
      if c not in ('0', "1", '2', '3', '4', '5', '6', '7', '8', '9', '«', '»', '–', "\""):
          str2 = str2 + c
  texts = str2
  str2 = ''

удалим знаки пунктуации

In [ ]:
import string
pattern = string.punctuation
for c in texts:
    if c not in pattern:
        str2 = str2 + c
    else:
        str2 = str2 + " "
texts = str2
str2 = ''

удалим стоп слова

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
  from nltk.corpus import stopwords
  russian_stopwords = stopwords.words("russian")
  for word in texts.split():
      if word not in (russian_stopwords):
          str2 = str2 + " " + word
  texts = str2

  str2 = ''
  for word in texts.split():
      if len(word) > 1:
          str2 = str2 + " " + word
  texts = str2
  print(texts)

 такое интернет вещей what is iot and iot security jpg мир стал цифровым сотовые телефоны обычное дело школах планшеты заменили тетради компании разрабатывают технологии нового поколения например автомобили водителя кажется связано собой особенно бизнесе будь автоматические системы безопасности ноутбуки количество устройств подключенных сети работающих вместе растет последним данным году интернету подключено около миллиарда устройств такого рода взамодействия специальный термин интернет вещей internet of things iot хотя термин обиходе лет сих пор полностью понимают самом деле означает почему важен бизнеса потребителей такое интернет вещей термин интернет вещей iot означает взаимодействие устройств подключенных интернету общающихся собой вероятно подумали таких вещах ноутбук умный телевизор iot охватывает значительно разнообразные устройства например электронику которая ранее подключена сети домашние холодильники кофемашины стиральные машины далее интернет вещей это необычные устройства

рассчитаем TF-IDF

In [ ]:
printTF_IDF(texts)

                        names  Word_Count    TF_IDF
5                       вещей          15  0.448011
0                         iot          12  0.358409
27                 устройства          12  0.358409
9                    интернет           7  0.209072
26                  устройств           7  0.209072
19                 обновления           6  0.179204
16                    которые           6  0.179204
13                  интернету           6  0.179204
10             интернет вещей           6  0.179204
29                        это           6  0.179204
8                      защиты           5  0.149337
7                      данные           5  0.149337
11                  интернета           5  0.149337
20                     пароль           5  0.149337
12            интернета вещей           5  0.149337
1              автоматические           4  0.119470
17                      могут           4  0.119470
18                   например           4  0.119470
4           

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#Суммаризации текстов 

Задача суммаризации текстов (автореферирование) - одна из ключевых, широко обсуждаемых задач NLP. Она состоит в сжатии больших объемов текста до связного краткого содержания, отражающего только основные идеи.


#Алгоритм TextRank
TextRank - это алгоритм, основанный на PageRank, который часто используется для извлечения ключевых слов и суммирования текста. 
PageRank (PR) - это алгоритм, используемый для расчета веса веб-страниц. 


In [ ]:
from gensim.summarization import summarize, keywords
print("Исходный текст:")
print(stexts)
print("Результат работы TextRank:")
print(summarize(str(stexts)))

Исходный текст:
Что такое интернет вещей? What is IoT and IoT Security.jpg Мир стал цифровым. Сотовые телефоны теперь обычное дело, в школах планшеты заменили тетради, а компании разрабатывают технологии нового поколения, например, автомобили без водителя.  Кажется, все связано между собой, особенно в бизнесе. Будь то автоматические системы безопасности или ноутбуки - количество устройств, подключенных к сети и работающих вместе, только растет. По последним данным, к 2020 году к Интернету будет подключено около 20,4 миллиарда устройств. Для такого рода «взамодействия» есть специальный термин - Интернет вещей (Internet of Things - IoT).  Хотя термин уже в обиходе больше 10 лет, не все до сих пор полностью понимают, что на самом деле он означает и почему он так важен для бизнеса и потребителей.   Что такое "интернет вещей"? Термин "интернет вещей" (IoT) - означает взаимодействие устройств, подключенных к Интернету и "общающихся" между собой. Вы, вероятно, сейчас подумали о таких вещах, к

#Алгоритм Rake
RAKE: Rapid Automatic Keyword Extraction Algorithm
Алгоритм RAKE извлекает ключевые слова с помощью основанного на разделителе подхода, чтобы идентифицировать ключевые слова кандидата и баллы их использующий совместные встречаемости слова, которые появляются в ключевых словах кандидата. Ключевые слова могут содержать несколько лексем. Кроме того, алгоритм RAKE также объединяет ключевые слова, когда они кажутся многократно, разделенными тем же разделителем слияния.


In [ ]:
!pip install rake_nltk
from rake_nltk import Metric, Rake
r = Rake(language="russian")
import nltk
nltk.download('punkt')
r.extract_keywords_from_text(stexts)
mas = r.get_ranked_phrases()
set2 = set()
for item in mas:
    if not "nan" in str(item).replace(" nan ", " "):
        set2.add(str(item).replace(" nan ", " "))
mas = list(set2)
print("Исходный текст:")
print(stexts)
print("Результат работы TextRammk:")
print(str(mas))

     |████████████████████████████████| 1.5 MB 7.2 MB/s 
     |████████████████████████████████| 749 kB 69.0 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Исходный текст:
Что такое интернет вещей? What is IoT and IoT Security.jpg Мир стал цифровым. Сотовые телефоны теперь обычное дело, в школах планшеты заменили тетради, а компании разрабатывают технологии нового поколения, например, автомобили без водителя.  Кажется, все связано между собой, особенно в бизнесе. Будь то автоматические системы безопасности или ноутбуки - количество устройств, подключенных к сети и работающих вместе, только растет. По последним данным, к 2020 году к Интернету будет подключено около 20,4 миллиарда устройств. Для такого рода «взамодействия» есть специальный термин - Интернет вещей (Internet of Things - IoT).  Хотя термин уже в обиходе больше 10 лет, не все до сих пор полностью понимают, что на самом деле он означает и почему он так важен для бизнеса и потребителей.   Что такое "интернет вещей"? Термин "интернет вещей" (IoT) - означает взаимодействие устр

#Тематическое моделирование
Тематическое моделирование — это метод извлечения тем из текста. Latent Dirichlet Allocation (LDA) — популярный алгоритм моделирования тем реализованные в том числе в пакете Gensim. Основная задача алгоритмов ТМ, заключается в том что бы полученные темы были хорошего качество, понятными, самозначимыми и разделенными. Достижение этих целей во многом зависит от качества предварительной обработки текста и стратегии поиска оптимального количества тем. 

##Импорт пакетов


In [ ]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Подход LDA к тематическому моделированию заключается в том, что каждый документ рассматривается как набор тем в определенной пропорции. И каждая тема как набор ключевых слов, опять же, в определенной пропорции.

После того, как вы укажете алгоритму количество тем, все, что он сделает, — это отобразит распределение тем в документах и распределение ключевых слов по темам.

Тема — это не что иное, как набор доминирующих ключевых слов. Просто взглянув на ключевые слова, вы сможете определить, о чем эта тема.

Ниже приведены ключевые факторы для получения хороших разделительных тем:

Качество обработки текста.
Разнообразие тем, о которых говорится в тексте.
Выбор алгоритма моделирование тем.
Количество тем, указанных в алгоритме.
Алгоритмы настройки параметров.

# Подготовим стоп-слова


In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

##Лемматизация
Лемматизация — это не что иное, как преобразование слова в его корневое слово. Например: лемма слова «machines» — это «machine». Аналогично, «walking» -> «walk», «mice» -> «mouse» и так далее.

In [ ]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
ltexts = ""
print(texts)
for word in texts.split():
    if len(str(word)) > 2:
        ltexts+=" " + morph.parse(word)[0].normal_form
print(ltexts)

     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 8.2 MB 12.0 MB/s 
 такое интернет вещей what is iot and iot security jpg мир стал цифровым сотовые телефоны обычное дело школах планшеты заменили тетради компании разрабатывают технологии нового поколения например автомобили водителя кажется связано собой особенно бизнесе будь автоматические системы безопасности ноутбуки количество устройств подключенных сети работающих вместе растет последним данным году интернету подключено около миллиарда устройств такого рода взамодействия специальный термин интернет вещей internet of things iot хотя термин обиходе лет сих пор полностью понимают самом деле означает почему важен бизнеса потребителей такое интернет вещей термин интернет вещей iot означает взаимодействие устройств подключенных интернету общающихся собой вероятно подумали таких вещах ноутбук умный телевизор iot охватывает значительно разнообразные устройства например электронику которая ране

##Создадим словарь и корпус.
Двумя основными входными данными для тематической модели LDA являются словарь (id2word) и корпус. Давайте создадим их.

In [ ]:
from nltk.util import ngrams
def make_bigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    bigrams = list(ngrams(token, 2))
    # print(bigrams)
    return bigrams
def make_trigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    trigrams = list(ngrams(token, 2))
    return trigrams

btexts = make_bigrams(texts) + make_trigrams(texts)
id2word = corpora.Dictionary(btexts)
corpus = [id2word.doc2bow(text) for text in btexts]
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('интернет', 1), ('такое', 1)]]

Или вы можете увидеть удобочитаемую форму самого корпуса.

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('интернет', 1), ('такое', 1)]]

##Построим тематическую модель
У нас есть все необходимое для обучения модели LDA. В дополнение к корпусу и словарю необходимо также указать количество тем.

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

##Просмотр тем в модели LDA
Вышеупомянутая модель LDA построена на 20 различных темах, где каждая тема представляет собой комбинацию ключевых слов, и каждое ключевое слово вносит определенный вес в тему.

Вы можете увидеть ключевые слова для каждой темы и вес (важность) каждого ключевого слова, используя lda_model.print_topics().

In [ ]:
lda_model.print_topics()

[(0,
  '0.027*"бизнеса" + 0.019*"вашего" + 0.018*"компания" + 0.018*"процессы" + 0.018*"имеет" + 0.018*"который" + 0.017*"приложения" + 0.015*"время" + 0.014*"сотрудника" + 0.014*"борьбы"'),
 (1,
  '0.095*"устройства" + 0.053*"защиты" + 0.045*"которые" + 0.039*"устройств" + 0.023*"могут" + 0.023*"своих" + 0.021*"возможности" + 0.020*"например" + 0.019*"функции" + 0.016*"устройство"'),
 (2,
  '0.041*"убедитесь" + 0.040*"устройствах" + 0.038*"интернет" + 0.036*"это" + 0.031*"безопасности" + 0.025*"интернету" + 0.024*"нужно" + 0.022*"подключенные" + 0.022*"свой" + 0.021*"кибератак"'),
 (3,
  '0.074*"вещей" + 0.047*"iot" + 0.042*"обновления" + 0.037*"интернета" + 0.022*"автоматические" + 0.022*"кибербезопасности" + 0.022*"решений" + 0.019*"предоставляет" + 0.019*"доступные" + 0.018*"использования"'),
 (4,
  '0.054*"данные" + 0.048*"пароль" + 0.038*"используйте" + 0.027*"использовать" + 0.021*"шифрование" + 0.021*"сквозное" + 0.021*"логин" + 0.021*"легче" + 0.015*"информацию" + 0.012*"данны

##Визуализация темы и ключевых слов
Теперь, когда модель LDA создана, следующим шагом является изучение созданных тем и связанных с ними ключевых слов. Нет лучшего инструмента, чем интерактивная диаграмма пакета pyLDAvis.

In [ ]:
!pip install pyLDAvis==2.1.2
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

     |████████████████████████████████| 1.6 MB 7.0 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=697c0cec935ad2b76f479145653ce71ce52498cf3e91eb11d0b2e9197d0a8005
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.033865  0.136174       1        1  21.742980
3     -0.244479  0.064938       2        1  21.505391
0     -0.060384 -0.268079       3        1  21.132615
2      0.209473 -0.054302       4        1  19.784089
4      0.129256  0.121269       5        1  15.834925, topic_info=           Term       Freq      Total Category  logprob  loglift
86   устройства  53.000000  53.000000  Default  30.0000  30.0000
2         вещей  41.000000  41.000000  Default  29.0000  29.0000
344      защиты  30.000000  30.000000  Default  28.0000  28.0000
305      данные  22.000000  22.000000  Default  27.0000  27.0000
5           iot  26.000000  26.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
353    надежных   4.290336   8.980710   Topic5  -4.5500   1.1042
325    защитный   3.563847   4.261183   Topic5  -4.7355   1.6642
326     продукт   3.563847   4.261183   Topic5  -4.7355   1.6642
324    украдено   3.563847   4.261183   Topic5  -4.7355   1.6642
273      разных   3.569954   8.188562   Topic5  -4.7338   1.0128

[234 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
139       1  0.887980              fi
5         2  0.972626             iot
138       1  0.887980              wi
34        2  0.931327  автоматические
428       2  0.877919   авторитетному
...     ...       ...             ...
421       2  0.877919          хотите
137       1  0.887980           число
374       5  0.958555      шифрование
245       3  0.781423     электронные
97        4  0.954225             это

[222 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3, 5])